In [1]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_excel("D:/从c盘转移的桌面/github/Python Financial Engineering/Financial Engineering Project2/rule_data.xlsx")
data.head()

,uid,oil_actv_dt,create_dt,total_oil_cnt,pay_amount_total,class_new,bad_ind,oil_amount,discount_amount,sale_amount,amount,pay_amount,coupon_amount,payment_coupon_amount,channel_code,oil_code,scene,source_app,call_source
0,A8217710,2018-08-19,2018-08-17,275.0,48295495.4,B,0,3308.56,1760081.0,1796001.0,1731081.0,8655401.0,1.0,1.0,1,3,2,0,3
1,A8217710,2018-08-19,2018-08-16,275.0,48295495.4,B,0,4674.68,2487045.0,2537801.0,2437845.0,12189221.0,1.0,1.0,1,3,2,0,3
2,A8217710,2018-08-19,2018-08-15,275.0,48295495.4,B,0,1873.06,977845.0,997801.0,961845.0,4809221.0,1.0,1.0,1,2,2,0,3
3,A8217710,2018-08-19,2018-08-14,275.0,48295495.4,B,0,4837.78,2526441.0,2578001.0,2484441.0,12422201.0,1.0,1.0,1,2,2,0,3
4,A8217710,2018-08-19,2018-08-13,275.0,48295495.4,B,0,2586.38,1350441.0,1378001.0,1328441.0,6642201.0,1.0,1.0,1,2,2,0,3


In [4]:
org_list = ['uid','create_dt','oil_actv_dt','class_new','bad_ind']
agg_list = ['oil_amount','discount_amount','sale_amount','amount','pay_amount','coupon_amount','payment_coupon_amount']
count_list = ['channel_code','oil_code','scene','source_app','call_source']

In [5]:
df=data[org_list].copy()
df[agg_list]=data[agg_list].copy()
df[count_list]=data[count_list].copy()
df.isna()

,uid,create_dt,oil_actv_dt,class_new,bad_ind,oil_amount,discount_amount,sale_amount,amount,pay_amount,coupon_amount,payment_coupon_amount,channel_code,oil_code,scene,source_app,call_source
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50604,False,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False
50605,False,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False
50606,False,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False
50607,False,True,False,False,False,True,True,True,True,True,True,True,False,False,False,False,False


In [6]:
def time_isna(x,y):
    if str(x)=='NaT':
        x=y
    return x

In [7]:
df2=df.sort_values(['uid','create_dt'],ascending=False)

df2['create_dt']=df2.apply(lambda x:time_isna(x.create_dt,x.oil_actv_dt),axis=1)

df2['dtn']=(df2.oil_actv_dt-df2.create_dt).apply(lambda x:x.days)

df=df2[df2['dtn']<180]

df.groupby('uid')['dtn'].count()

uid
A10000481              1
A1000232               3
A1000455               1
A10010042             15
A1001014               1
                      ..
B96436391972106043     1
B96436391976480892     1
B96436391977217468     1
B96436391984693397     1
B96436391985035703     1
Name: dtn, Length: 11099, dtype: int64

In [8]:
base=df[org_list].copy()
base['dtn']=df['dtn']
base=base.sort_values(['uid','create_dt'],ascending=False)
base=base.drop_duplicates(['uid'],keep='first')

In [9]:
gn=pd.DataFrame()
for i in agg_list:
    tp=df.groupby('uid').apply(lambda x:len(x[i])).reset_index()
    tp.columns=['uid',i+'_cnt']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    tp =df.groupby('uid').apply(lambda x:np.where(x[i]>0,1,0).sum()).reset_index()
    tp.columns=['uid',i+'_num']
    
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')

In [10]:
import warnings
warnings.filterwarnings("ignore")

gn=pd.DataFrame()
for i in agg_list:
    tp=df.groupby('uid').apply(lambda df:len(df[i])).reset_index()
    tp.columns=['uid',i+'_cnt']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    
    tp=df.groupby('uid').apply(lambda df:np.where(df[i]>0,1,0).sum()).reset_index()
    tp.columns=['uid',i+"_num"]
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
    #sum  
    tp=df.groupby('uid').apply(lambda df:np.nansum(df[i])).reset_index()
    tp.columns=['uid',i+'_tot']
    if gn.empty:
        gn=tp
    else:
        gn=pd.merge(gn,tp,on='uid',how='left')
        
    #average
    tp = df.groupby('uid').apply(lambda df:np.nanmean(df[i])).reset_index()
    tp.columns = ['uid',i + '_avg']
    if gn.empty:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = 'uid',how = 'left')
    #max
    tp = df.groupby('uid').apply(lambda df:np.nanmax(df[i])).reset_index()
    tp.columns = ['uid',i + '_max']
    if gn.empty:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = 'uid',how = 'left')
    #mini
    tp = df.groupby('uid').apply(lambda df:np.nanmin(df[i])).reset_index()
    tp.columns = ['uid',i + '_min']
    if gn.empty:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = 'uid',how = 'left')
    #sd
    tp = df.groupby('uid').apply(lambda df:np.nanvar(df[i])).reset_index()
    tp.columns = ['uid',i + '_var']
    if gn.empty:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = 'uid',how = 'left')
    #range
    tp = df.groupby('uid').apply(lambda df:np.nanmax(df[i]) -np.nanmin(df[i]) ).reset_index()
    tp.columns = ['uid',i + '_ran']
    if gn.empty:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = 'uid',how = 'left')

gc=pd.DataFrame()
for i in count_list:
    tp=df.groupby('uid').apply(lambda x:len(set(x[i]))).reset_index()
    tp.columns=['uid',i+'_dstc']
    if gc.empty:
        gc=tp
    else:
        gc=pd.merge(gc,tp,on='uid',how='left')


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\2740599671.py:25: RuntimeWarning: Mean of empty slice
  tp = df.groupby('uid').apply(lambda df:np.nanmean(df[i])).reset_index()
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\2740599671.py:32: RuntimeWarning: All-NaN axis encountered
  tp = df.groupby('uid').apply(lambda df:np.nanmax(df[i])).reset_index()
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\2740599671.py:39: RuntimeWarning: All-NaN axis encountered
  tp = df.groupby('uid').apply(lambda df:np.nanmin(df[i])).reset_index()
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\2740599671.py:46: RuntimeWarning: Degrees of freedom <= 0 for slice.
  tp = df.groupby('uid').apply(lambda df:np.nanvar(df[i])).reset_index()
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\2740599671.py:53: RuntimeWarning: All-NaN axis encountered
  tp = df.groupby('uid').apply(lambda df:np.nanmax(df[i]) -np.nanmin(df[i]) ).reset_index()
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_11732\274059967

In [11]:
fn=pd.merge(base,gn,on='uid')
fn=pd.merge(fn,gc,on='uid')
fn.shape

fn=fn.fillna(0)
fn.head()

,uid,create_dt,oil_actv_dt,class_new,bad_ind,dtn,oil_amount_cnt,oil_amount_num,oil_amount_tot,oil_amount_avg,...,payment_coupon_amount_avg,payment_coupon_amount_max,payment_coupon_amount_min,payment_coupon_amount_var,payment_coupon_amount_ran,channel_code_dstc,oil_code_dstc,scene_dstc,source_app_dstc,call_source_dstc
0,B96436391985035703,2018-10-08,2018-10-08,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
1,B96436391984693397,2018-10-11,2018-10-11,E,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
2,B96436391977217468,2018-10-17,2018-10-17,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
3,B96436391976480892,2018-09-28,2018-09-28,B,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
4,B96436391972106043,2018-10-19,2018-10-19,A,0,0,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1


In [12]:
x=fn.drop(['uid','oil_actv_dt','create_dt','bad_ind','class_new'],axis=1)
y=fn.bad_ind.copy()

In [13]:
from sklearn import tree
dtree=tree.DecisionTreeRegressor(max_depth=2,min_samples_leaf=500,min_samples_split=5000)
dtree=dtree.fit(x,y)

In [16]:
!pip install pydotplus
import pydotplus 
from IPython.display import Image
from six import StringIO
import os

In [17]:
with open("dt.dot", "w") as f:
    tree.export_graphviz(dtree, out_file=f)
dot_data = StringIO()

In [18]:
tree.export_graphviz(dtree, out_file=dot_data,
                         feature_names=x.columns,
                         class_names=['bad_ind'],
                         filled=True, rounded=True,
                         special_characters=True)

In [20]:
group_1 = fn.loc[(fn.amount_tot>48077.5)&(fn.sale_amount_cnt>3.5)].copy()
group_1['level'] = 'past_A'
group_2 = fn.loc[(fn.amount_tot>48077.5)&(fn.sale_amount_cnt<=3.5)].copy()
group_2['level'] = 'past_B'
group_3 = fn.loc[fn.amount_tot<=48077.5].copy()
group_3['level'] = 'past_C'

group_3.bad_ind.sum()/group_3.shape[0]

0.07390733445882573